In [1]:
import argparse
import scanpy as sc
import plotnine as plt9
import pandas as pd
import numpy as np
#import matplotlib as plt
import os
import matplotlib.pyplot as plt

In [2]:
#fpath1="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/data/gut-freeze003/ti-cd_healthy-auto_processed_v001-labels_freeze_v003/analysis/adata/cell_prob_filter_0pt5/ti-cd_healthy-auto_processed_v001-labels_freeze_v003-old_samples-processed.h5ad"
#adata1=sc.read(fpath1)  

#fpath2="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/data/gut-freeze003/ti-cd_healthy-auto_processed_v001-labels_freeze_v003/analysis/adata/cell_prob_filter_0pt5/ti-cd_healthy-auto_processed_v001-labels_freeze_v003-new_samples-processed.h5ad"
#adata2=sc.read(fpath2)


fpath1="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/data/gut-freeze003/auto-annot-121/adata/anderson_ti_freeze003_001-MT_100-cohort_Discovery.h5ad"
adata1=sc.read(fpath1)  

fpath2="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/data/gut-freeze003/auto-annot-121/adata/anderson_ti_freeze003_001-MT_100-cohort_Replication.h5ad"
adata2=sc.read(fpath2) 


fpath3="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/data/gut-freeze003/auto-annot-121/adata/anderson_ti_freeze003_001-MT_100-cohort_Full.h5ad"
adata3=sc.read(fpath3)

In [3]:
#adata3[adata3.obs['sanger_sample_id']=="OTARscRNA9294504",]

In [4]:
f="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/repo/sc_nextflow/studies/gut-freeze003/data-gene_scores.tsv"
gene_scores=pd.read_table(f, sep='\t')

inflammation_score_smilie=gene_scores[gene_scores.grouping_id=='inflammation_signature__SmillieCS_31348891'].ensembl_gene_id
apoptosis_score_smilie=gene_scores[gene_scores.grouping_id=='HALLMARK_APOPTOSIS'].ensembl_gene_id


In [5]:
f="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/repo/sc_nextflow/studies/gut-freeze003/data-variable_gene_filter.tsv"
gene_scores1=pd.read_table(f, sep='\t')

mitochondrial_score=gene_scores1[gene_scores1.gene_type=='mitochondrial'].gene_symbol.to_list()


mitochondrial_score1=np.array(adata1[:,adata1.var['gene_symbols'].isin(list(mitochondrial_score))].layers['log1p_cp10k'].todense())
mitochondrial_score2=np.array(adata2[:,adata2.var['gene_symbols'].isin(list(mitochondrial_score))].layers['log1p_cp10k'].todense())
mitochondrial_score3=np.array(adata3[:,adata3.var['gene_symbols'].isin(list(mitochondrial_score))].layers['log1p_cp10k'].todense())

mitochondrial_score1=np.mean(mitochondrial_score1, axis=1)
mitochondrial_score2=np.mean(mitochondrial_score2, axis=1)
mitochondrial_score3=np.mean(mitochondrial_score3, axis=1)

In [6]:
crypt_axis_genes_moor_top_entero = ["APOA4", "APOC3", "ALPI", "PKIB", "PMP22", "SLC28A2"]

crypt_axis_genes_ensemble = adata1.var[adata1.var['gene_symbols'].isin(crypt_axis_genes_moor_top_entero)].index.values
sc.tl.score_genes(adata1, crypt_axis_genes_ensemble)
adata1.obs['crypt_villus_score_moor_top_entero'] = adata1.obs['score']

crypt_axis_genes_ensemble = adata2.var[adata2.var['gene_symbols'].isin(crypt_axis_genes_moor_top_entero)].index.values
sc.tl.score_genes(adata2, crypt_axis_genes_ensemble)
adata2.obs['crypt_villus_score_moor_top_entero'] = adata2.obs['score']

crypt_axis_genes_ensemble = adata1.var[adata3.var['gene_symbols'].isin(crypt_axis_genes_moor_top_entero)].index.values
sc.tl.score_genes(adata3, crypt_axis_genes_ensemble)
adata3.obs['crypt_villus_score_moor_top_entero'] = adata3.obs['score']

In [7]:
f="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/manuscript/ti_atlas_code/figures/data/Reactome/INTERFERON_GAMMA_SIGNALING.tsv"
gene_scores=pd.read_table(f, sep='\t')
interferon_gamma=np.unique(gene_scores['MoleculeName'].str.split(" ", expand=True)[1].values)
interferon_gamma=adata3.var[adata3.var['gene_symbols'].isin(interferon_gamma)].index.tolist()


f="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/manuscript/ti_atlas_code/figures/data/Reactome/INTERFERON_ALPHA_BETA_SIGNALING.tsv"
gene_scores=pd.read_table(f, sep='\t')
interferon_alpha_beta=np.unique(gene_scores['MoleculeName'].str.split(" ", expand=True)[1].values)
interferon_alpha_beta=adata3.var[adata3.var['gene_symbols'].isin(interferon_alpha_beta)].index.tolist()

In [8]:
fpath="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/manuscript/ti_atlas_code_helper/costimulatory_coinhibitory.csv"
df6=pd.read_table(fpath, sep=',', header=0)
df7=df6[df6['gene_type']=="T cell receptor"]
costim_receptor_symbol=df7[df7['function']=="co-stimulatory"].gene_symbol.tolist()
#coinh_receptor=df7[df7['function']=="co-inhibitory"].gene_symbol.tolist()
costim_receptor=adata3.var[adata3.var['gene_symbols'].isin(costim_receptor_symbol)].index.tolist()

df8=df6[df6['gene_type']=="APC ligand"]
costim_ligand_symbol=df8[df8['function']=="co-stimulatory"].gene_symbol.tolist()
#coinh_ligand=df8[df8['function']=="co-inhibitory"].gene_symbol.tolist()
costim_ligand=adata3.var[adata3.var['gene_symbols'].isin(costim_ligand_symbol)].index.tolist()

costimr_mean1=np.array(adata1[:,adata1.var['gene_symbols'].isin(list(costim_receptor_symbol))].layers['log1p_cp10k'].todense())
costimr_mean2=np.array(adata2[:,adata2.var['gene_symbols'].isin(list(costim_receptor_symbol))].layers['log1p_cp10k'].todense())
costimr_mean3=np.array(adata3[:,adata3.var['gene_symbols'].isin(list(costim_receptor_symbol))].layers['log1p_cp10k'].todense())

costimr_mean1=np.mean(costimr_mean1, axis=1)
costimr_mean2=np.mean(costimr_mean2, axis=1)
costimr_mean3=np.mean(costimr_mean3, axis=1)


costiml_mean1=np.array(adata1[:,adata1.var['gene_symbols'].isin(list(costim_ligand_symbol))].layers['log1p_cp10k'].todense())
costiml_mean2=np.array(adata2[:,adata2.var['gene_symbols'].isin(list(costim_ligand_symbol))].layers['log1p_cp10k'].todense())
costiml_mean3=np.array(adata3[:,adata3.var['gene_symbols'].isin(list(costim_ligand_symbol))].layers['log1p_cp10k'].todense())

costiml_mean1=np.mean(costiml_mean1, axis=1)
costiml_mean2=np.mean(costiml_mean2, axis=1)
costiml_mean3=np.mean(costiml_mean3, axis=1)

In [9]:
#gene_scores[['grouping_id', "description"]].drop_duplicates()

In [10]:
#sc.tl.score_genes(adata1, inflammation_score_smilie, score_name='inflammation_score')
#sc.tl.score_genes(adata2, inflammation_score_smilie, score_name='inflammation_score')
#sc.tl.score_genes(adata3, inflammation_score_smilie, score_name='inflammation_score')

sc.tl.score_genes(adata1, apoptosis_score_smilie, score_name='apoptosis_score')
sc.tl.score_genes(adata2, apoptosis_score_smilie, score_name='apoptosis_score')
sc.tl.score_genes(adata3, apoptosis_score_smilie, score_name='apoptosis_score')


sc.tl.score_genes(adata1, interferon_gamma, score_name='interferon_gamma_score')
sc.tl.score_genes(adata2, interferon_gamma, score_name='interferon_gamma_score')
sc.tl.score_genes(adata3, interferon_gamma, score_name='interferon_gamma_score')


sc.tl.score_genes(adata1, interferon_alpha_beta, score_name='interferon_alpha_beta_score')
sc.tl.score_genes(adata2, interferon_alpha_beta, score_name='interferon_alpha_beta_score')
sc.tl.score_genes(adata3, interferon_alpha_beta, score_name='interferon_alpha_beta_score')

sc.tl.score_genes(adata1, costim_receptor, score_name='costim_receptor_score')
sc.tl.score_genes(adata2, costim_receptor, score_name='costim_receptor_score')
sc.tl.score_genes(adata3, costim_receptor, score_name='costim_receptor_score')

sc.tl.score_genes(adata1, costim_ligand, score_name='costim_ligand_score')
sc.tl.score_genes(adata2, costim_ligand, score_name='costim_ligand_score')
sc.tl.score_genes(adata3, costim_ligand, score_name='costim_ligand_score')



KeyboardInterrupt: 

In [ ]:
SLCO2A1=np.array(adata1[:,adata1.var['gene_symbols'].isin(list(["SLCO2A1"]))].layers['log1p_cp10k'].todense())
SLCO2A1_mean=np.mean(SLCO2A1, axis=1)

SLCO2A2=np.array(adata2[:,adata2.var['gene_symbols'].isin(list(["SLCO2A1"]))].layers['log1p_cp10k'].todense())
SLCO2A2_mean=np.mean(SLCO2A2, axis=1)

SLCO2A3=np.array(adata3[:,adata3.var['gene_symbols'].isin(list(["SLCO2A1"]))].layers['log1p_cp10k'].todense())
SLCO2A3_mean=np.mean(SLCO2A3, axis=1)

In [11]:
fpath="/home/ubuntu/ubuntu/sc_ibd_project/ti_atlas/clean_annotation/data-clean_annotation-full.csv"
df5=pd.read_table(fpath, sep=',')
df5=df5[["category", "label__machine_retired", "label"]]

In [12]:
mhc=["HLA-A", "HLA-B", "HLA-C", "HLA-E", "HLA-F","HLA-G","HLA-DRA","HLA-DRB5","HLA-DRB1",  "HLA-DQA1",  "HLA-DQA2", "HLA-DQB1", "HLA-DQB2",
       "HLA-DMA","HLA-DMB", "HLA-DOA", "HLA-DOB", "HLA-DPA1", "HLA-DPB1"]

In [23]:
mhc_d=np.array(adata1[:,adata1.var['gene_symbols'].isin(mhc)].layers['log1p_cp10k'].todense())
mhc_r=np.array(adata2[:,adata2.var['gene_symbols'].isin(mhc)].layers['log1p_cp10k'].todense())
mhc_f=np.array(adata3[:,adata3.var['gene_symbols'].isin(mhc)].layers['log1p_cp10k'].todense())

In [24]:
mhc_d=pd.DataFrame(mhc_d, columns=adata1[:,adata1.var['gene_symbols'].isin(mhc)].var.gene_symbols.values.astype('str').tolist())
mhc_d['label__machine_retired']=adata1.obs['label__machine'].reset_index().label__machine
mhc_d['disease_status']=adata1.obs['disease_status'].reset_index().disease_status
mhc_d['sanger_sample_id']=adata1.obs['sanger_sample_id'].reset_index().sanger_sample_id
mhc_d['sex']=adata1.obs['sex'].reset_index().sex
mhc_d['age_imputed']=adata1.obs['age_imputed'].reset_index().age_imputed
mhc_d['inflammation_status']=adata1.obs['inflammation_status'].reset_index().inflammation_status
mhc_d['apoptosis_score']=adata1.obs['apoptosis_score'].reset_index().apoptosis_score
mhc_d['interferon_gamma_score']=adata1.obs['interferon_gamma_score'].reset_index().interferon_gamma_score
mhc_d['interferon_alpha_beta_score']=adata1.obs['interferon_alpha_beta_score'].reset_index().interferon_alpha_beta_score
mhc_d['costim_receptor_score']=adata1.obs['costim_receptor_score'].reset_index().costim_receptor_score
mhc_d['costim_ligand_score']=adata1.obs['costim_ligand_score'].reset_index().costim_ligand_score
mhc_d['costim_receptor_mean']=costimr_mean1
mhc_d['costim_ligand_mean']=costiml_mean1
mhc_d['mitochondrial_mean']=mitochondrial_score1
mhc_d['crypt_villus_score_moor_top_entero']=adata1.obs['crypt_villus_score_moor_top_entero'].reset_index().crypt_villus_score_moor_top_entero
mhc_d=pd.melt(mhc_d, id_vars=['label__machine_retired','disease_status', 'sanger_sample_id',
                              'sex', 'age_imputed', 'inflammation_status', "apoptosis_score", 
                              'interferon_gamma_score', 'interferon_alpha_beta_score', 'costim_receptor_score', 
                              'costim_ligand_score', 'costim_receptor_mean','costim_ligand_mean', 'mitochondrial_mean'])
mhc_d=mhc_d.rename(columns={'variable':'gene_symbol', 'value':'log1p_cp10k'})
mhc_d['cohort']='Discovery'
   

mhc_r=pd.DataFrame(mhc_r, columns=adata2[:,adata2.var['gene_symbols'].isin(mhc)].var.gene_symbols.values.astype('str').tolist())
mhc_r['label__machine_retired']=adata2.obs['label__machine'].reset_index().label__machine
mhc_r['disease_status']=adata2.obs['disease_status'].reset_index().disease_status
mhc_r['sanger_sample_id']=adata2.obs['sanger_sample_id'].reset_index().sanger_sample_id
mhc_r['sex']=adata2.obs['sex'].reset_index().sex
mhc_r['age_imputed']=adata2.obs['age_imputed'].reset_index().age_imputed
mhc_r['inflammation_status']=adata2.obs['inflammation_status'].reset_index().inflammation_status
mhc_r['apoptosis_score']=adata2.obs['apoptosis_score'].reset_index().apoptosis_score
mhc_r['interferon_gamma_score']=adata2.obs['interferon_gamma_score'].reset_index().interferon_gamma_score
mhc_r['interferon_alpha_beta_score']=adata2.obs['interferon_alpha_beta_score'].reset_index().interferon_alpha_beta_score
mhc_r['costim_receptor_score']=adata2.obs['costim_receptor_score'].reset_index().costim_receptor_score
mhc_r['costim_ligand_score']=adata2.obs['costim_ligand_score'].reset_index().costim_ligand_score
mhc_r['costim_receptor_mean']=costimr_mean2
mhc_r['costim_ligand_mean']=costiml_mean2
mhc_r['mitochondrial_mean']=mitochondrial_score2
mhc_r['crypt_villus_score_moor_top_entero']=adata2.obs['crypt_villus_score_moor_top_entero'].reset_index().crypt_villus_score_moor_top_entero
mhc_r=pd.melt(mhc_r, id_vars=['label__machine_retired','disease_status', 'sanger_sample_id',
                             'sex', 'age_imputed', 'inflammation_status', "apoptosis_score", 
                              'interferon_gamma_score', 'interferon_alpha_beta_score', 'costim_receptor_score', 'costim_ligand_score', 
                              'costim_receptor_mean','costim_ligand_mean', 'mitochondrial_mean'])
mhc_r=mhc_r.rename(columns={'variable':'gene_symbol', 'value':'log1p_cp10k'})
mhc_r['cohort']='Replication'


mhc_f=pd.DataFrame(mhc_f, columns=adata3[:,adata3.var['gene_symbols'].isin(mhc)].var.gene_symbols.values.astype('str').tolist())
mhc_f['label__machine_retired']=adata3.obs['label__machine'].reset_index().label__machine
mhc_f['disease_status']=adata3.obs['disease_status'].reset_index().disease_status
mhc_f['sanger_sample_id']=adata3.obs['sanger_sample_id'].reset_index().sanger_sample_id
mhc_f['sex']=adata3.obs['sex'].reset_index().sex
mhc_f['age_imputed']=adata3.obs['age_imputed'].reset_index().age_imputed
mhc_f['inflammation_status']=adata3.obs['inflammation_status'].reset_index().inflammation_status
mhc_f['apoptosis_score']=adata3.obs['apoptosis_score'].reset_index().apoptosis_score
mhc_f['interferon_gamma_score']=adata3.obs['interferon_gamma_score'].reset_index().interferon_gamma_score
mhc_f['interferon_alpha_beta_score']=adata3.obs['interferon_alpha_beta_score'].reset_index().interferon_alpha_beta_score
mhc_f['costim_receptor_score']=adata3.obs['costim_receptor_score'].reset_index().costim_receptor_score
mhc_f['costim_ligand_score']=adata3.obs['costim_ligand_score'].reset_index().costim_ligand_score
mhc_f['costim_receptor_mean']=costimr_mean3
mhc_f['costim_ligand_mean']=costiml_mean3
mhc_f['mitochondrial_mean']=mitochondrial_score3
mhc_f['crypt_villus_score_moor_top_entero']=adata3.obs['crypt_villus_score_moor_top_entero'].reset_index().crypt_villus_score_moor_top_entero
mhc_f=pd.melt(mhc_f, id_vars=['label__machine_retired','disease_status', 'sanger_sample_id',
                              'sex', 'age_imputed', 'inflammation_status', "apoptosis_score", 
                              'interferon_gamma_score', 'interferon_alpha_beta_score', 'costim_receptor_score', 'costim_ligand_score', 
                              'costim_receptor_mean','costim_ligand_mean', 'mitochondrial_mean'])
mhc_f=mhc_f.rename(columns={'variable':'gene_symbol', 'value':'log1p_cp10k'})
mhc_f['cohort']='Full'


In [25]:
df4=pd.concat([mhc_d, mhc_r, mhc_f])

In [26]:
df4

,label__machine_retired,disease_status,sanger_sample_id,sex,age_imputed,inflammation_status,apoptosis_score,interferon_gamma_score,interferon_alpha_beta_score,costim_receptor_score,costim_ligand_score,costim_receptor_mean,costim_ligand_mean,mitochondrial_mean,gene_symbol,log1p_cp10k,cohort
0,Paneth_cell,cd,5892STDY8039553,M,36.0,mild,0.056689,0.006760,0.055392,-0.074177,-0.080126,0.025919,0.000000,3.766616,HLA-F,0.000000,Discovery
1,Paneth_cell,cd,5892STDY8039553,M,36.0,mild,0.074970,-0.019384,0.014099,-0.119725,-0.083033,0.005717,0.012653,4.235732,HLA-F,0.000000,Discovery
2,Paneth_cell,cd,5892STDY8039553,M,36.0,mild,0.053731,-0.006661,0.055909,-0.069727,-0.069728,0.016577,0.000000,3.815475,HLA-F,0.052242,Discovery
3,Paneth_cell,cd,5892STDY8039553,M,36.0,mild,0.086769,0.014839,0.063496,-0.125230,-0.105943,0.011548,0.008886,3.873500,HLA-F,0.053314,Discovery
4,Paneth_cell,cd,5892STDY8039553,M,36.0,mild,0.071727,0.011625,0.068911,-0.088429,-0.084877,0.016908,0.000000,3.768450,HLA-F,0.103977,Discovery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8838235,B_cell,cd,Pilot_study_of_dissociation_methods_for_human_...,M,30.0,mild,0.020439,0.142533,-0.009132,-0.073125,0.144702,0.000000,0.214613,0.594312,crypt_villus_score_moor_top_entero,-0.064384,Full
8838236,B_cell,cd,Pilot_study_of_dissociation_methods_for_human_...,M,30.0,mild,0.022686,0.155770,0.142721,-0.086802,-0.056092,0.000000,0.000000,0.000000,crypt_villus_score_moor_top_entero,-0.043628,Full
8838237,B_cell_plasma_IgA_CD38plus,cd,Pilot_study_of_dissociation_methods_for_human_...,M,30.0,mild,0.004341,0.415961,0.163759,-0.099058,-0.061164,0.000000,0.000000,0.838232,crypt_villus_score_moor_top_entero,-0.095349,Full
8838238,T_cell_CD4_CD40LGplus_2,cd,Pilot_study_of_dissociation_methods_for_human_...,M,30.0,mild,0.049529,0.111739,0.076603,0.551340,-0.068819,0.625628,0.000000,3.971538,crypt_villus_score_moor_top_entero,-0.087773,Full


In [27]:
df6=pd.merge(df4,df5, on="label__machine_retired")
df6=df6.drop(columns=['label__machine_retired'])


In [28]:
df6.to_csv("data/absolute_expression_mhc12.csv", index=False)

In [20]:
#gene_scores[['grouping_id', 'study_id','description']].drop_duplicates()

In [20]:
df6

,disease_status,sanger_sample_id,sex,age_imputed,inflammation_status,apoptosis_score,interferon_gamma_score,interferon_alpha_beta_score,costim_receptor_score,costim_ligand_score,costim_receptor_mean,costim_ligand_mean,mitochondrial_mean,gene_symbol,log1p_cp10k,cohort,crypt_villus_score_moor_top_entero,category,label
0,cd,5892STDY8039553,M,36.0,mild,0.056689,0.006760,0.055392,-0.074177,-0.080126,0.025919,0.000000,3.766616,HLA-F,0.000000,Discovery,-0.023436,Secretory,Paneth cell
1,cd,5892STDY8039553,M,36.0,mild,0.074970,-0.019384,0.014099,-0.119725,-0.083033,0.005717,0.012653,4.235732,HLA-F,0.000000,Discovery,-0.024302,Secretory,Paneth cell
2,cd,5892STDY8039553,M,36.0,mild,0.053731,-0.006661,0.055909,-0.069727,-0.069728,0.016577,0.000000,3.815475,HLA-F,0.052242,Discovery,-0.019446,Secretory,Paneth cell
3,cd,5892STDY8039553,M,36.0,mild,0.086769,0.014839,0.063496,-0.125230,-0.105943,0.011548,0.008886,3.873500,HLA-F,0.053314,Discovery,-0.002548,Secretory,Paneth cell
4,cd,5892STDY8039553,M,36.0,mild,0.071727,0.011625,0.068911,-0.088429,-0.084877,0.016908,0.000000,3.768450,HLA-F,0.103977,Discovery,-0.018706,Secretory,Paneth cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16980012,healthy,OTARscRNA9342934,F,36.0,uninflamed,0.298698,0.848577,0.457151,-0.226430,0.736456,0.000000,0.876271,2.680210,HLA-DPB1,4.561958,Full,NaN,Myeloid,Mac intermediate (2)
16980013,healthy,OTARscRNA9997689,M,41.0,uninflamed,0.299257,0.498456,0.424511,-0.142058,0.156505,0.000000,0.249603,3.236347,HLA-DPB1,0.000000,Full,NaN,Myeloid,Mac intermediate (2)
16980014,healthy,OTARscRNA9997691,F,59.0,uninflamed,0.159229,0.525886,0.431439,0.083053,0.112770,0.225359,0.221111,2.488513,HLA-DPB1,2.028235,Full,NaN,Myeloid,Mac intermediate (2)
16980015,healthy,OTARscRNA9997693,F,75.0,uninflamed,0.135418,0.348452,0.187422,-0.155684,-0.038052,0.000000,0.000000,1.587042,HLA-DPB1,0.000000,Full,NaN,Myeloid,Mac intermediate (2)
